In [1]:
import requests
from concurrent.futures import ThreadPoolExecutor

# Requests to the Model on torch serve

In [1]:
def get_response(request_data):
#     print(f'request_data {request_data}')
    return requests.post('http://0.0.0.0:8080/predictions/bert_multiclass', data=request_data)

def _response_data(response_list):
    clfs_size = 12
    count_200 = [0 if res.status_code == 200 else 1 for res in response_list]
    print(f'count_200 sum {sum(count_200)}')    
    return [res.json() if res.status_code == 200 else [0] * clfs_size for res in response_list]

def query(data, **kwargs):
    """
    Sends the batches to server
    :param data: input texts

    :return: list of replies
    """   
    
    BSIZE = 32
    num_news = len(data)
    if num_news == 0:
        return []

    responses = []
    request_list = [{"data": sent} for sent in data]

        
    
    n_batches = len(request_list) // BSIZE

    for i, slc in enumerate(range(n_batches + 1)):
        req_slice = request_list[slc * BSIZE:(slc + 1) * BSIZE]
        with ThreadPoolExecutor(max_workers=BSIZE) as pool:
            responses += list(pool.map(get_response, req_slice))
            
    responses_data = _response_data(responses)
    num_proper_replies = sum([res.status_code == 200 for res in responses])
    
    
    assert len(responses_data) == num_news, \
        f"Something went wrong with the server, got {len(responses_data)} instead of {num_news}"

    return responses_data

